In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
import torch
torch.cuda.device_count()

/home/tbadmaev/new_conda/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


0

In [9]:
import diffusion
from diffusion.models import BertLMHeadModel as OurMaskedAttentionBert

In [10]:
from diffusion.models.score_estimator import ScoreEstimator

In [11]:
from transformers import BertConfig, BertLMHeadModel

bert_conf = BertConfig('bert-base-uncased')
score_estimator = ScoreEstimator(bert_conf)
from torch_ema import ExponentialMovingAverage
ema = ExponentialMovingAverage(score_estimator.parameters(), 0)

ckpt = torch.load('../experiments/ce_masked/step=10000.ckpt', map_location='cpu')

In [12]:
ema.load_state_dict(ckpt['callbacks']['EMACallback'])
st_dict = type(ckpt['state_dict'])()
patt = 'score_estimator'
for k, v in ckpt['state_dict'].items():
    if patt in k:
        st_dict[k[len(patt)+1:]] = v
score_estimator.load_state_dict(st_dict)

ValueError: Tried to `load_state_dict()` with the wrong number of parameters in the saved state.

: 

In [15]:
from diffusion.dataset.glue_data import SST2Dataset
from torch.utils.data import DataLoader

train_dataset = SST2Dataset(train=True)
valid_dataset = SST2Dataset(train=False)


def get_val_loader():
    return DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

def get_train_loader():
    return DataLoader(train_dataset, batch_size=256, shuffle=False, drop_last=False)

Found cached dataset glue (/home/tbadmaev/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/tbadmaev/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [22]:
from tqdm.auto import tqdm
from diffusion.utils import dict_to_device


def print_accuracy(loader, bert_model):
    total_true_pred = 0
    total_label_pred = 0
    total_size = 0
    total_word_count = 0

    total_label_from_all = 0

    bar = tqdm(loader)
    for batch in bar:
        batch = dict_to_device(batch, 'cuda')
        count = torch.sum(batch['attention_mask'], dim=-1).long()
        logits = bert_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask']).logits
        # 0 - [CLS], 1 - label, count-1 - [SEP]
        pred_label_from_all = torch.argmax(logits[:, 1], dim=-1)
        labels_cat = batch['labels'].view(-1) * 2748 + (1 - batch['labels'].view(-1)) * 2053
        total_label_from_all += torch.sum((pred_label_from_all == labels_cat).float())


        pred_label_logits = logits[:, 1, [2053, 2748]]
        pred_label = torch.argmax(pred_label_logits, dim=-1)
        total_label_pred += torch.sum((pred_label == batch['labels'].view(-1)).float())
        total_size += len(pred_label)

        all_pred_ids = torch.argmax(logits, dim=-1)
        correct = (all_pred_ids == batch['input_ids'])
        total_word_count += torch.sum(count - 3)
        for sent, end_idx in zip(correct, count):
            total_true_pred += torch.sum(sent[2:end_idx])

        bar.set_description(f'label_acc: {total_label_pred / total_size:.5f}, \
                            text_recon_acc: {total_true_pred / total_word_count:.5f}, \
                            label_acc_from_all: {total_label_from_all/ total_size:.5f}')

    return total_label_pred / total_size, total_true_pred / total_word_count, total_label_from_all/ total_size

##### Our masked attention bert on validation

In [23]:
print_accuracy(get_val_loader(), our_bert)

  0%|          | 0/4 [00:00<?, ?it/s]

(tensor(0.6250, device='cuda:0'),
 tensor(0.9393, device='cuda:0'),
 tensor(0.2317, device='cuda:0'))

##### Default bert on validation

In [24]:
print_accuracy(get_val_loader(), h_bert)

  0%|          | 0/4 [00:00<?, ?it/s]

(tensor(0.7110, device='cuda:0'),
 tensor(0.9455, device='cuda:0'),
 tensor(0.3567, device='cuda:0'))

##### Our masked attention bert on train

In [25]:
print_accuracy(get_train_loader(), our_bert)

  0%|          | 0/264 [00:00<?, ?it/s]

(tensor(0.7652, device='cuda:0'),
 tensor(0.8824, device='cuda:0'),
 tensor(0.3382, device='cuda:0'))

##### Default bert on train

In [26]:
print_accuracy(get_train_loader(), h_bert)

  0%|          | 0/264 [00:00<?, ?it/s]

(tensor(0.8031, device='cuda:0'),
 tensor(0.8955, device='cuda:0'),
 tensor(0.4452, device='cuda:0'))

In [29]:
batch = next(iter(get_train_loader()))
batch = dict_to_device(batch, 'cuda')

logits = h_bert(input_ids=batch['input_ids'], attention_mask=batch['attention_mask']).logits
pred_ids = torch.argmax(logits, dim=-1)
pred_label_from_all = torch.argmax(logits[:, 1], dim=-1)
labels_cat = batch['labels'].view(-1) * 2748 + (1 - batch['labels'].view(-1)) * 2053

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


input_sent = tokenizer.batch_decode(batch['input_ids'])
input_sent[:5]

['[CLS] no hide new secretions from the parental units [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] no contains no wit, only labored gags [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] yes that loves its characters and communicates something rather beautiful about human nature [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [30]:
pred_sent = tokenizer.batch_decode(pred_ids)
pred_sent[:5]

['. no hide new secretions from the parental....... no no no.. no no... no no no new new or or the....... no no no.. no no new.. no no no no no no hide new itemsions from the other....... no',
 '... no wit, only labored gag....... no wit no no. no.... it no humor any nor and no wit. and. and... no. no. no no all and.. no no - it - much much wit nor only short short....... no',
 '.. that loves its characters and communicates something rather beautiful about human....... it it it.. it it... it it and communicate, and communicate of. its. and... it it it.. it it yes yes. it it loves its characters and communicate with something and deeply about....',
 '.. remains utterly satisfied to remain the same....... he no.... no... he he he,, and and,. forever.. forever.. he he he. he he.... he he he he he he,,, and and, forever...... he he',
 '. no on the worst revenge - of - the - nerds cliches the filmmakers could dr make....... no the the some. - - - the - - punk and movie and the ever could. e